

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

--------------------------------------------------------------------------------------------------------------------

Los préstamos personales son una lucrativa fuente de ingresos para los bancos. El tipo de interés típico de un préstamo a dos años en el Reino Unido ronda el 10%. Puede que esto no parezca mucho, pero solo en septiembre de 2022 los consumidores del Reino Unido pidieron prestado alrededor de £1,500 millones, lo que significaría aproximadamente £300 millones en intereses generados por los bancos durante dos años.

Le pidieron que trabajara con un banco para limpiar los datos que recopilaron como parte de una campaña de marketing reciente, cuyo objetivo era lograr que los clientes solicitaran un préstamo personal. Planean realizar más campañas de marketing en el futuro, por lo que le gustaría que se asegure de que se ajuste a la estructura específica y los tipos de datos que especifican para que luego puedan usar los datos limpios que usted proporcione para configurar una base de datos PostgreSQL, que almacenará los datos de esta campaña. datos y permitir que los datos de futuras campañas se importen fácilmente.

Le han proporcionado un archivo csv llamado "bank_marketing.csv", que deberá limpiar, reformatear y dividir los datos, guardando tres archivos csv finales. Específicamente, los tres archivos deben tener los nombres y contenidos que se describen a continuación:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [37]:
import pandas as pd
import numpy as np

# Start coding here...

In [38]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
credit_default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64
mortgage
--------------
mortgage
yes        21576
no         18622
unknown      990
Name: count, dtype: int64
previous_outcome
--------------
previous_outcome
nonexistent    35563
failure         4252
success         1373
Name: count, dtype: int64
campaign_outcome
--------------
campaign_outcome
no     36548
yes     4640
Name: count, dtype: int64


In [39]:
df.head(5)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [40]:
client= df[["client_id", "age", "job", "marital", "education", "credit_default", "mortgage"]]
campaign = df[["client_id", "number_contacts", "contact_duration", "previous_campaign_contacts", "previous_outcome",  "campaign_outcome"]]
economic = df[["client_id", "cons_price_idx", "euribor_three_months"]]



In [41]:
client.loc[:, 'job'] = client['job'].str.replace('.', '_')

#client['job'] = client['job'].str.replace('.', '_')
client.loc[:,'education'] = client['education'].str.replace('.','_') 

client['education'].replace('unknown', np.NaN)


0                   basic_4y
1                high_school
2                high_school
3                   basic_6y
4                high_school
                ...         
41183    professional_course
41184    professional_course
41185      university_degree
41186    professional_course
41187    professional_course
Name: education, Length: 41188, dtype: object

In [44]:
# Verificar los valores únicos después del mapeo
# Crear un diccionario de mapeo para convertir los valores
mapping = {'yes': 1, 'no': 0, 'unknown': 0}

# Aplicar el mapeo a la columna 'credit_default' y convertir a tipo booleano
client['credit_default'] = client['credit_default'].map(mapping).astype(bool)

# Verificar los valores únicos después del mapeo
print(client['credit_default'].unique())



[False  True]


C:\Users\dmcar\AppData\Local\Temp\ipykernel_23976\1134955616.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client['credit_default'] = client['credit_default'].map(mapping).astype(bool)


In [43]:
campaign["previous_outcome"] = campaign["previous_outcome"].astype("bool")

campaign["campaign_outcome"] = campaign["campaign_outcome"].astype("bool")

#esto es un ERROR ASI LA CARGUE campaign['last_contact_date'].astype("date")
campaign['last_contact_date'] = None
campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date'], format="YYYY-MM-DD")

campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  bool          
 5   campaign_outcome            41188 non-null  bool          
 6   last_contact_date           0 non-null      datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(4)
memory usage: 1.6 MB


C:\Users\dmcar\AppData\Local\Temp\ipykernel_23976\668579701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign["previous_outcome"] = campaign["previous_outcome"].astype("bool")
C:\Users\dmcar\AppData\Local\Temp\ipykernel_23976\668579701.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign["campaign_outcome"] = campaign["campaign_outcome"].astype("bool")
C:\Users\dmcar\AppData\Local\Temp\ipykernel_23976\668579701.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic